In [2]:
import pickle
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import DataLoader, TensorDataset
import math
import copy

## Load data

This uses the discharge-voltage curves at each cycle as raw data. Dimensionality is $(100,1000)$, with 100 cycles and 1000 voltage steps at which discharge is measured

In [3]:
data = pickle.load(open(r'../Data/discharge_curves.pkl', 'rb'))

In [4]:
x = np.stack(data["discharge_curves"].values) # reshape data
y = data["cycle_life"].values

In [5]:
np.shape(x) # 124 batteries, 99 cycles, discharge measured at 1000 voltage steps per cycle

(124, 99, 1000)

In [6]:
np.shape(y)

(124,)

In [7]:
# convert to tensors and concatenate features and labels
x = torch.Tensor(x)
y = torch.Tensor(y)

In [8]:
position_embeddings = nn.Parameter(torch.randn(len(x), 99, 1000))

# Add position embedding into patch embedding
input_embeddings = x + position_embeddings

## Split into train/test sets

In [9]:
test_ind = np.array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32,
       34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66,
       68, 70, 72, 74, 76, 78, 80, 82, 83])

train_ind = np.array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33,
       35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63, 65, 67,
       69, 71, 73, 75, 77, 79, 81])

secondary_ind = np.array([ 84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,
        97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109,
       110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122,
       123])

In [10]:
train = TensorDataset(torch.Tensor(x[train_ind]),torch.Tensor(y[train_ind]))
test = TensorDataset(torch.Tensor(x[test_ind]),torch.Tensor(y[test_ind]))
stest = TensorDataset(torch.Tensor(x[secondary_ind]),torch.Tensor(y[secondary_ind]))

## Transformer model

In [11]:
input_embeddings.size()

torch.Size([124, 99, 1000])

In [12]:
# parameters
d_model = 1000
nhead = 8
batch_size = 3
nepoch = 1
batch_first = True

In [13]:
num_heads = 8
num_layers = 12

# Define Transformer encoders' stack
transformer_encoder_layer = nn.TransformerEncoderLayer(
           d_model=d_model, nhead=num_heads,
           dim_feedforward=int(d_model * 4),
           dropout=0.1)
transformer_encoder = nn.TransformerEncoder(
           encoder_layer=transformer_encoder_layer,
           num_layers=num_layers)

# Forward pass
output_embeddings = transformer_encoder(input_embeddings[:10])
print(f' Output embedding size: {output_embeddings.size()}')

 Output embedding size: torch.Size([10, 99, 1000])


In [14]:
regressor = nn.Linear(d_model, 1) 
output_regression = regressor(torch.sum(output_embeddings, dim = 1))
output_regression

tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], grad_fn=<AddmmBackward0>)

In [20]:
class FullModel(torch.nn.Module):

    def __init__(self):
        super(FullModel, self).__init__()

        self.transformer = transformer_encoder
        torch.nn.init.xavier_uniform(self.transformer.weight)
        self.linear = regressor
        torch.nn.init.xavier_uniform(self.linear.weight)

    def forward(self, x):
        x = self.transformer(x)
        x = self.linear(x)
        return x

In [21]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# Load nodel, loss function, and optimizer
model = FullModel().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Load batch image
train_dataset = TensorDataset(torch.Tensor(x[train_ind]),torch.Tensor(y[train_ind]))
# train_dataloader = DataLoader(train_dataset, num_workers=1, batch_size=batch_size, shuffle=True)

# # Fine tuning loop
for i in range(1):
    total_acc_train = 0
    total_loss_train = 0.0

    for train_input, train_label in train_dataset[:10]:
        output = model(train_input.to(device))
        print(output)
        # loss = criterion(output, train_label.to(device))
        # acc = (output.argmax(dim=1) == train_label.to(device)).sum().item()
        # total_acc_train += acc
        # total_loss_train += loss.item()

        # loss.backward()
        # optimizer.step()
        # optimizer.zero_grad()

    # print(f'Epochs: {i + 1} | Loss: {total_loss_train / len(train_dataset): .3f} | Accuracy: {total_acc_train / len(train_dataset): .3f}')


# Hyperparameters
EPOCHS = 10
LEARNING_RATE = 1e-4
BATCH_SIZE = 8

train_dataloader

AttributeError: 'TransformerEncoder' object has no attribute 'weight'